In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff




In [2]:
yandex_world = pd.read_csv('yandex_world.csv')
yandex_world.rename(columns = {'Дата':'Date','Страна':'Country','Заражений':'Confirmed','Выздоровлений':'Recovered',
                               'Смертей':'Deaths','Смертей за день':'Day_deaths',
                               'Заражений за день':'Day_confirmed','Выздоровлений за день':'Day_recovered'},inplace = True)
for i in range(yandex_world.shape[0]):
    a= yandex_world.Date[i].split('.')
    yandex_world.Date[i]=datetime.date(int(a[2]),int(a[1]),int(a[0]))

yandex_world.tail(20)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
43177,Япония,2020-08-20,59900,45110,1157,1172,1324,9
43178,Япония,2020-08-21,60949,46264,1175,1049,1154,18
43179,Япония,2020-08-22,61916,47418,1179,967,1154,4
43180,Япония,2020-08-23,62658,48319,1188,742,901,9
43181,Япония,2020-08-24,63158,49089,1201,500,770,13
43182,Япония,2020-08-25,63888,50092,1219,730,1003,18
43183,Япония,2020-08-26,64779,51407,1230,891,1315,11
43184,Япония,2020-08-27,65653,52542,1241,874,1135,11
43185,Япония,2020-08-28,66499,53691,1251,846,1149,10
43186,Япония,2020-08-29,67353,54883,1271,854,1192,20


In [3]:
df3_temp = yandex_world.groupby('Date').sum()
df3_temp['Country']='Весь мир'

y5= pd.concat([yandex_world,df3_temp.reset_index()],axis = 0)
y5.reset_index()
yandex_world = y5
yandex_world.tail(10)

,Country,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths
221,Весь мир,2020-08-30,25222709,16616995,846353,226253,208436,3901
222,Весь мир,2020-08-31,25484767,16818436,850535,262058,201441,4182
223,Весь мир,2020-09-01,25749642,17073236,857015,264875,254800,6480
224,Весь мир,2020-09-02,26031410,17291874,863028,281768,218638,6013
225,Весь мир,2020-09-03,26304856,17512585,868733,273446,220711,5705
226,Весь мир,2020-09-04,26617971,17726336,874639,313115,213751,5906
227,Весь мир,2020-09-05,26881547,17927195,879577,263576,200859,4938
228,Весь мир,2020-09-06,27103845,18137310,883339,222298,210115,3762
229,Весь мир,2020-09-07,27337760,18336112,892646,233915,198802,9307
230,Весь мир,2020-09-08,27570742,18524391,897383,232982,188279,4737


In [4]:
# dt = datetime.datetime.today()
# today = datetime.date(dt.year,dt.month,dt.day)
# ystday = datetime.date(today.year,today.month,today.day-1)
# yst2day = datetime.date(today.year,today.month,today.day-2)
# yst3day = datetime.date(today.year,today.month,today.day-3)

# print(today)
# print(ystday)
# print(yst2day)
# print(yst3day)
myday = list(yandex_world.tail(1).Date)[0]

print(yandex_world.iloc[-1,0])
print('Выбран день: ',myday)

Весь мир
Выбран день:  2020-09-08


In [5]:
X = pd.DataFrame()
country_dict = {}
yandex_world['Rt'] = 1
yandex_world['Rt2'] = 1
yandex_world['Rate_conf'] = 0
yandex_world['Rate_ill'] = 0

temp_df = yandex_world.groupby(['Country','Date']).sum()

# Ниже enumerate  для составления словаря из имен регионов {i:j} , для расчетов можно было использовать только j

for i,j in enumerate(temp_df.index.get_level_values(0).unique()):
    temp_df2 =  temp_df.loc[j]
    temp1 = temp_df2['Day_confirmed'].rolling(4).sum()/(temp_df2["Day_confirmed"].rolling(8).sum()-

                                                              temp_df2["Day_confirmed"].rolling(4).sum())
    temp2 = temp_df2['Confirmed'].rolling(4).sum()/(temp_df2['Confirmed'].rolling(8).sum()-

                                                              temp_df2['Confirmed'].rolling(4).sum())
    temp_df2['Rt'] = round(temp1,3)
    temp_df2['Rt2'] = round(temp2,3)
    temp_df2['Country'] = j
    temp_df2['Remaining_ill']=temp_df2['Confirmed']-temp_df2['Recovered']-temp_df2['Deaths']
    temp_df2['MA7_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=7).mean()
    temp_df2['MA14_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=14).mean()
    temp_df2['MA7_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=7).mean()
    temp_df2['MA14_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=14).mean()
    
    temp_df2['Change_dayconf'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_dayconf'].iloc[k]=temp_df2['Day_confirmed'].iloc[k]-temp_df2['Day_confirmed'].iloc[k-1]
    temp_df2['Change_remill'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_remill'].iloc[k]=temp_df2['Remaining_ill'].iloc[k]-temp_df2['Remaining_ill'].iloc[k-1]
    
    

   #Генератор DF  на каждый регион(сидит в цикле):
   # globals()['x' + str(i)] = temp_df2.reset_index()
    
    X = pd.concat([X,temp_df2.reset_index()],axis = 0)
    country_dict[i]=j
    
print(country_dict.items())
    
X = X.reset_index(drop=True)
X.tail()


dict_items([(0, 'Австралия'), (1, 'Австрия'), (2, 'Азербайджан'), (3, 'Албания'), (4, 'Алжир'), (5, 'Ангола'), (6, 'Андорра'), (7, 'Антигуа и Барбуда'), (8, 'Аргентина'), (9, 'Армения'), (10, 'Афганистан'), (11, 'Багамские острова'), (12, 'Бангладеш'), (13, 'Барбадос'), (14, 'Бахрейн'), (15, 'Беларусь'), (16, 'Белиз'), (17, 'Бельгия'), (18, 'Бенин'), (19, 'Болгария'), (20, 'Боливия'), (21, 'Босния и Герцеговина'), (22, 'Ботсвана'), (23, 'Бразилия'), (24, 'Бруней'), (25, 'Буркина-Фасо'), (26, 'Бурунди'), (27, 'Бутан'), (28, 'Великобритания'), (29, 'Венгрия'), (30, 'Венесуэла'), (31, 'Весь мир'), (32, 'Восточный Тимор'), (33, 'Вьетнам'), (34, 'Габон'), (35, 'Гаити'), (36, 'Гайана'), (37, 'Гамбия'), (38, 'Гана'), (39, 'Гватемала'), (40, 'Гвинея'), (41, 'Гвинея-Бисау'), (42, 'Германия'), (43, 'Гондурас'), (44, 'Гренада'), (45, 'Греция'), (46, 'Грузия'), (47, 'ДР Конго'), (48, 'Дания'), (49, 'Джибути'), (50, 'Доминика'), (51, 'Доминиканская Республика'), (52, 'Египет'), (53, 'Замбия'), (54,

,Date,Confirmed,Recovered,Deaths,Day_confirmed,Day_recovered,Day_deaths,Rt,Rt2,Rate_conf,Rate_ill,Country,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
43423,2020-09-04,70866,60093,1352,588,1893,18,0.900,1.035,0,0,Япония,9421,623.857143,708.357143,10498.285714,11498.000000,-71,-1323
43424,2020-09-05,71467,61099,1361,601,1006,9,0.968,1.035,0,0,Япония,9007,587.714286,682.214286,10185.142857,11190.000000,13,-414
43425,2020-09-06,71918,61730,1366,451,631,5,1.015,1.035,0,0,Япония,8822,565.714286,661.428571,9877.857143,10880.785714,-150,-185
43426,2020-09-07,72213,62190,1377,295,460,11,0.834,1.033,0,0,Япония,8646,545.285714,646.785714,9575.857143,10579.214286,-156,-176
43427,2020-09-08,72724,62936,1398,511,746,21,0.752,1.031,0,0,Япония,8390,528.714286,631.142857,9316.285714,10280.142857,216,-256


In [6]:
# Добавляем колонки расчитанные сколько доля их в процентах от суммарного показателя в мире
mask = X.Country == 'Весь мир'
temp_df = X[mask][['Date','Confirmed','Remaining_ill']]
temp_df.index = temp_df.Date
temp_df.drop(columns =['Date'], inplace = True)
print(temp_df.tail(2))
# print(temp_df.index.get_level_values(0))
# temp_df.loc[datetime.date(2020,6,15),'Россия'].Confirmed
# for i in temp_df.index.get_level_values(0).unique()[-2:]:
#     print(temp_df.loc[i,'Россия'].Confirmed)
world_date = list(temp_df.index)
world_conf = list(temp_df.Confirmed)
world_ill = list(temp_df.Remaining_ill)
world_dict_conf = dict(zip(world_date,world_conf))
world_dict_ill = dict(zip(world_date,world_ill))

print(len(X))
print(X.shape)
print(X.loc[0,'Date'])
for i in range(len(X)):
    X.loc[i,'Rate_conf']=100*X.loc[i,'Confirmed']/world_dict_conf.get(X.loc[i,'Date'])
    X.loc[i,'Rate_ill']=100*X.loc[i,'Remaining_ill']/world_dict_ill.get(X.loc[i,'Date'])


Confirmed  Remaining_ill
Date                                
2020-09-07   27337760        8109002
2020-09-08   27570742        8148968
43428
(43428, 19)
2020-01-22


In [7]:
# Проверка расчетов процентных показателей
mask = (X.Date>=datetime.date(2020,6,13))&(X.Country.isin(['Весь мир','Россия']))

temp=X[mask]
temp[['Date','Confirmed','Country','Rate_conf','Rate_ill']].tail(20)

,Date,Confirmed,Country,Rate_conf,Rate_ill
29779,2020-08-20,939833,Россия,4.144738,2.314787
29780,2020-08-21,944671,Россия,4.116922,2.271603
29781,2020-08-22,949531,Россия,4.090767,2.238049
29782,2020-08-23,954328,Россия,4.075276,2.263089
29783,2020-08-24,959016,Россия,4.056090,2.285446
29784,2020-08-25,963655,Россия,4.034412,2.256850
29785,2020-08-26,968297,Россия,4.005573,2.216065
29786,2020-08-27,972972,Россия,3.978956,2.171644
29787,2020-08-28,977730,Россия,3.952908,2.134503
29788,2020-08-29,982573,Россия,3.930849,2.107445


In [8]:
X.to_csv('W.csv',index=False)